In [1]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.
from trl import DPOTrainer, DPOConfig
from datasets import Dataset


# from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

In [2]:
# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")

In [3]:
config_data = {
    "general": {
        "data_path": "../data/", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": "digit82/kobart-summarization", # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": "./" # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
    },
    "tokenizer": {
        "encoder_max_len": 512,
        "decoder_max_len": 100,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#']
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 20,
        "learning_rate": 1e-5,
        "per_device_train_batch_size": 128,
        "per_device_eval_batch_size": 32,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 2,
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 100,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.001,
        "report_to": "wandb" # (선택) wandb를 사용할 때 설정합니다.
    },
    # (선택) wandb 홈페이지에 가입하여 얻은 정보를 기반으로 작성합니다.
    "wandb": {
        "project": "dialogSUM",
        "name": "baselinedpo001",
    },
    "inference": {
        "ckt_path": "model ckt path", # 사전 학습이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 100,
        "num_beams": 4,
        "batch_size" : 32,
        # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    }
}

In [4]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
config_path = "./config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

### 3) Configuration 불러오기

In [5]:
# 저장된 config 파일을 불러옵니다.
config_path = "./config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
pprint(loaded_config)

{'general': {'data_path': '../data/',
             'model_name': 'digit82/kobart-summarization',
             'output_dir': './'},
 'inference': {'batch_size': 32,
               'ckt_path': 'model ckt path',
               'early_stopping': True,
               'generate_max_length': 100,
               'no_repeat_ngram_size': 2,
               'num_beams': 4,
               'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>'],
               'result_path': './prediction/'},
 'tokenizer': {'bos_token': '<s>',
               'decoder_max_len': 100,
               'encoder_max_len': 512,
               'eos_token': '</s>',
               'special_tokens': ['#Person1#',
                                  '#Person2#',
                                  '#Person3#',
                                  '#PhoneNumber#',
                                  '#Address#',
                                  '#PassportNumber#']},
 'training': {'do_eval': True,
              'do_train': True,
              

### 4) 데이터 불러와서 확인해보기
- 실험에서 쓰일 데이터를 load하여 데이터의 구조와 내용을 살펴보겠습니다.
- Train, dev, test 순서대로 12457, 499, 250개 씩 데이터가 구성되어 있습니다.

In [6]:
# config에 저장된 데이터 경로를 통해 train과 validation data를 불러옵니다.
data_path = loaded_config['general']['data_path']

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(data_path,'train_all.csv'))
train_df.tail()

,fname,ground_dialogue_ko,dialogue_en_trans,dialogue_en_trans2ko,groundtruth_summary,solarAPI_summary_from_ground_dialogue,solarAPI_summary_from_dialogue_en_trans2ko,topic,NER
12452,train_12455,#Person1#: 안녕하세요. 혹시 맨체스터에서 오신 Mr. Green 맞으신가요...,#Person1#: Hello. Are you Mr. Green from Manch...,#Person1#: 안녕하세요. 맨체스터에서 오신 그린 씨 맞으시죠? \n#Per...,Tan Ling은 흰머리와 수염이 특징인 Mr. Green을 맞이하여 호텔로 안내합...,#Person1#이 맨체스터에서 온 #Person2#를 Phoenix 호텔 스위트로...,": \n#Person1#은 맨체스터에서 온 #Person2#를 정중히 맞이하며, ...",호텔 예약 확인,"Manchester, Mr. Green, Yellow River Import and..."
12453,train_12456,"#Person1#: Mister Ewing이 우리 회의장에 4시에 오라고 했지, 맞...",#Person1#: Mr. Ewing said to come to our meeti...,#Person1#: 유잉 씨가 오후 4시에 회의실로 오라고 했죠? \n#Perso...,#Person1#과 #Person2#는 Mister Ewing의 요청에 따라 회의장...,미스터 Ewing은 #Person1#과 #Person2#에게 4시에 회의장에 늦지 ...,유잉 씨는 오후 4시 회의실에서 East York 지점 방문객과 회의를 진행하도록 ...,회의 참석 방법,"Mister Ewing, 이스트 요크 지점"
12454,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person1#: How can I assist you today? \n#Per...,#Person1#: 오늘 어떻게 도와드릴까요? \n#Person2#: 차를 렌트하...,#Person2#는 #Person1#의 도움으로 5일 동안 소형차를 대여합니다.,#Person2#가 도심 이동을 위해 소형차를 5일간 대여하려 함. #Person1...,"#Person1#은 렌트 가능한 차량을 안내하며, #Person2#는 도시 단독 운...",소형차 렌트,"Named Entities: 소형차, 대형차, 중형차, 운전면허증, 신용카드"
12455,train_12458,#Person1#: 너 오늘 좀 기분 안 좋아 보인다? 무슨 일 있어?\n#Pers...,#Person1#: You look a bit down today. Is somet...,#Person1#: 오늘 좀 기운이 없어 보이네. 무슨 일 있어? \n#Perso...,#Person2#의 어머니가 직장을 잃으셨다. #Person2#는 어머니가 우울해하...,"#Person1#이 #Person2#의 우울한 기분을 언급하며 사정을 묻자, #Pe...","#Person1#은 #Person2#의 어머니가 실직했다는 소식을 듣고 위로하며, ...",실업률 여성 일자리 문제,"엄마, 지역 커뮤니티"
12456,train_12459,"#Person1#: 엄마, 나 다음 주 토요일에 이모부네 가족 보러 가는데, 오늘 ...","#Person1#: Mom, I’m going to visit my uncle’s ...","#Person1#: 엄마, 다음 주 토요일에 이모부 가족한테 놀러 갈 건데, 오늘 ...",#Person1#은 다음 주 토요일에 이모부네 가족을 방문하기 위해 짐을 싸야 하는...,#Person1#이 다음 주 토요일 이모부네 가족을 방문하기 위해 오늘 짐을 쌀 계...,"#Person1#이 다음 주 토요일 이모부 가족 방문 시 가방 준비를 요청하자, #...",여행 준비 및 날씨 확인,"이모부네, 사촌 수잔"


In [7]:
# validation data의 구조와 내용을 확인합니다.
val_df = pd.read_csv(os.path.join(data_path,'dev.csv'))
val_df.tail()

,fname,dialogue,summary,topic
494,dev_495,#Person1#: 새해가 되니까 나도 새 출발을 하기로 했어.\n#Person2#...,#Person1#은 새해에 담배를 끊고 커밍아웃 하기로 결심했습니다. #Person...,새해 결심
495,dev_496,#Person1#: 너 Joe랑 결혼했지?\n#Person2#: Joe? 무슨 말이...,"#Person1#은 #Person2#가 Joe와 결혼했다고 생각하지만, #Perso...",사랑과 결혼 오해
496,dev_497,"#Person1#: 어떻게 도와드릴까요, 아줌마?\n#Person2#: 제 차에서 ...","#Person2#의 차에서 소리가 나며, 브레이크 수리가 필요한 상황입니다. #Pe...",차량 소음 및 수리
497,dev_498,"#Person1#: 여보세요, 아마존 고객 서비스입니다. 어떻게 도와드릴까요?\n#...",#Person2#가 아마존 고객 서비스에 전화하여 아마존에서 구매한 책에 53페이지...,책 페이지 누락
498,dev_499,#Person1#: 벌써 여름이 다가오다니 믿기지 않아. \n#Person2#: 맞...,"#Person2#는 여름방학 동안 파티에서 일하는 회사에서 일하며, 주로 음식 준비...",여름방학 일자리


## 1. 데이터 가공 및 데이터셋 클래스 구축
- csv file 을 불러와서 encoder 와 decoder의 입력형태로 가공해줍니다.
- 가공된 데이터를 torch dataset class 로 구축하여 모델에 입력가능한 형태로 만듭니다.

In [8]:
# 데이터 전처리를 위한 클래스로, 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성합니다.
class Preprocess:
    def __init__(self,
            bos_token: str,
            eos_token: str,
        ) -> None:

        self.bos_token = bos_token
        self.eos_token = eos_token

    @staticmethod
    # 실험에 필요한 컬럼을 가져옵니다.
    def make_set_as_df(file_path, is_train = True):
        if is_train:
            df = pd.read_csv(file_path)
            col_dlg = 'ground_dialogue_ko' if 'ground_dialogue_ko' in df.columns else 'dialogue'
            col_sum = 'groundtruth_summary' if 'groundtruth_summary' in df.columns else 'summary'
            df['prompt'] = df[col_dlg]
            df['chosen'] = df[col_sum]
            df['rejected'] = df['solarAPI_summary_from_ground_dialogue']
            train_df = df[['fname', col_dlg, col_sum,'rejected']]
            return train_df
        else:
            df = pd.read_csv(file_path)
            col_dlg = 'ground_dialogue_ko' if 'ground_dialogue_ko' in df.columns else 'dialogue'
            col_sum = 'groundtruth_summary' if 'groundtruth_summary' in df.columns else 'summary'
            df['prompt'] = df[col_dlg]
            df['chosen'] = df[col_sum]
            df['rejected'] = df['solarAPI_summary_from_ground_dialogue']
            test_df = df[['fname', col_dlg, col_sum,'rejected']]
            return test_df

    # BART 모델의 입력, 출력 형태를 맞추기 위해 전처리를 진행합니다.
    def make_input(self, dataset,is_test = False):
        if is_test:
            encoder_input = dataset['dialogue']
            decoder_input = [self.bos_token] * len(dataset['dialogue'])
            return encoder_input.tolist(), list(decoder_input)
        else:
            col_dlg = 'ground_dialogue_ko' if 'ground_dialogue_ko' in dataset.columns else 'dialogue'
            col_sum = 'groundtruth_summary' if 'groundtruth_summary' in dataset.columns else 'summary'
            encoder_input = dataset[col_dlg]
            decoder_rejected = dataset['rejected'].apply(lambda x : self.bos_token + str(x))
            decoder_input = dataset[col_sum].apply(lambda x : self.bos_token + str(x)) # Ground truth를 디코더의 input으로 사용하여 학습합니다.
            decoder_output = dataset[col_sum].apply(lambda x : str(x) + self.eos_token)
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist(), decoder_rejected.tolist()


In [9]:

def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    # 1) 데이터 파일 경로
    train_file_path = os.path.join(data_path,'train_all.csv')
    val_file_path = os.path.join(data_path,'dev_all.csv')

    # 2) CSV 읽고 필요한 컬럼만 DataFrame 형태로 준비 (컬럼명 환경에 맞게 조정)
    train_data = preprocessor.make_set_as_df(train_file_path)
    val_data = preprocessor.make_set_as_df(val_file_path)

    print('-'*30)
    print(f'train_data sample:\n{train_data.iloc[0]}')
    print(f'val_data sample:\n{val_data.iloc[0]}')
    print('-'*30)

    # 3) 전처리: 각 데이터프레임에서 인코더 입력, 디코더 입력, 디코더 출력, 디코더 리젝션 생성
    encoder_input_train, decoder_input_train, decoder_output_train, decoder_input_train_reject = preprocessor.make_input(train_data)
    encoder_input_val, decoder_input_val, decoder_output_val, decoder_input_val_reject = preprocessor.make_input(val_data)

    # 4) 토큰화 시 주의점: return_tensors=None 으로 리스트 반환받아야 나중에 Dataset 변환 가능
    tokenized_encoder_inputs = tokenizer(
        encoder_input_train,
        padding=True,
        truncation=True,
        max_length=config['tokenizer']['encoder_max_len'],
        add_special_tokens=True,
        return_tensors=None
    )
    tokenized_decoder_inputs = tokenizer(
        decoder_input_train,
        padding=True,
        truncation=True,
        max_length=config['tokenizer']['decoder_max_len'],
        add_special_tokens=True,
        return_tensors=None
    )
    tokenized_decoder_outputs = tokenizer(
        decoder_output_train,
        padding=True,
        truncation=True,
        max_length=config['tokenizer']['decoder_max_len'],
        add_special_tokens=True,
        return_tensors=None
    )
    tokenized_decoder_input_train_reject = tokenizer(
        decoder_input_train_reject,
        padding=True,
        truncation=True,
        max_length=config['tokenizer']['decoder_max_len'],
        add_special_tokens=True,
        return_tensors=None
    )

    # 5) Hugging Face Dataset 생성: 딕셔너리 키 이름은 데이터 및 모델에 따라 다를 수 있음
    train_dataset = Dataset.from_dict({
        "input_ids": tokenized_encoder_inputs["input_ids"],
        "prompt": encoder_input_train,
        "attention_mask": tokenized_encoder_inputs["attention_mask"],
        "decoder_input_ids": tokenized_decoder_inputs["input_ids"],
        "decoder_attention_mask": tokenized_decoder_inputs["attention_mask"],
        "labels": tokenized_decoder_outputs["input_ids"],
        "chosen": decoder_input_train,
        'rejected': decoder_input_train_reject,

    })

    # 6) 검증 데이터도 동일 방식 처리
    val_tokenized_encoder_inputs = tokenizer(
        encoder_input_val,
        padding=True,
        truncation=True,
        max_length=config['tokenizer']['encoder_max_len'],
        add_special_tokens=True,
        return_tensors=None
    )
    val_tokenized_decoder_inputs = tokenizer(
        decoder_input_val,
        padding=True,
        truncation=True,
        max_length=config['tokenizer']['decoder_max_len'],
        add_special_tokens=True,
        return_tensors=None
    )
    val_tokenized_decoder_outputs = tokenizer(
        decoder_output_val,
        padding=True,
        truncation=True,
        max_length=config['tokenizer']['decoder_max_len'],
        add_special_tokens=True,
        return_tensors=None
    )
    val_tokenized_decoder_input_reject = tokenizer(
        decoder_input_val_reject,
        padding=True,
        truncation=True,
        max_length=config['tokenizer']['decoder_max_len'],
        add_special_tokens=True,
        return_tensors=None
    )

    val_dataset = Dataset.from_dict({
        "input_ids": val_tokenized_encoder_inputs["input_ids"],
        "prompt": encoder_input_val,
        "attention_mask": val_tokenized_encoder_inputs["attention_mask"],
        "decoder_input_ids": val_tokenized_decoder_inputs["input_ids"],
        "decoder_attention_mask": val_tokenized_decoder_inputs["attention_mask"],
        "labels": val_tokenized_decoder_outputs["input_ids"],
        "chosen": decoder_input_val,
        'rejected': decoder_input_val_reject,
        })

    print('-'*30)
    print(f'Train dataset example:\n{train_dataset[0]}')
    print(f'Validation dataset example:\n{val_dataset[0]}')
    print('-'*30)

    return train_dataset, val_dataset

In [10]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
def compute_metrics(config,tokenizer,pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    print('-'*150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[1]}")
    print(f"GOLD: {replaced_labels[1]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[2]}")
    print(f"GOLD: {replaced_labels[2]}")

    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)

    # ROUGE 점수 중 F-1 score를 통해 평가합니다.
    result = {key: value["f"] for key, value in results.items()}
    return result

In [11]:
# 학습을 위한 trainer 클래스와 매개변수를 정의합니다.
def load_trainer_for_train(config,generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset):
    print('-'*10, 'Make training arguments', '-'*10,)
    # set training args
    training_args = Seq2SeqTrainingArguments(
                output_dir=config['general']['output_dir'], # model output directory
                overwrite_output_dir=config['training']['overwrite_output_dir'],
                num_train_epochs=config['training']['num_train_epochs'],  # total number of training epochs
                learning_rate=config['training']['learning_rate'], # learning_rate
                per_device_train_batch_size=config['training']['per_device_train_batch_size'], # batch size per device during training
                per_device_eval_batch_size=config['training']['per_device_eval_batch_size'],# batch size for evaluation
                warmup_ratio=config['training']['warmup_ratio'],  # number of warmup steps for learning rate scheduler
                weight_decay=config['training']['weight_decay'],  # strength of weight decay
                lr_scheduler_type=config['training']['lr_scheduler_type'],
                optim =config['training']['optim'],
                gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
                eval_strategy=config['training']['evaluation_strategy'], # evaluation strategy to adopt during training
                save_strategy =config['training']['save_strategy'],
                save_total_limit=config['training']['save_total_limit'], # number of total save model.
                fp16=config['training']['fp16'],
                load_best_model_at_end=config['training']['load_best_model_at_end'], # 최종적으로 가장 높은 점수 저장
                seed=config['training']['seed'],
                logging_dir=config['training']['logging_dir'], # directory for storing logs
                logging_strategy=config['training']['logging_strategy'],
                predict_with_generate=config['training']['predict_with_generate'], #To use BLEU or ROUGE score
                generation_max_length=config['training']['generation_max_length'],
                do_train=config['training']['do_train'],
                do_eval=config['training']['do_eval'],
                report_to=config['training']['report_to'] # (선택) wandb를 사용할 때 설정합니다.
            )

    training_args = DPOConfig(
        output_dir=config['general']['output_dir'], # model output directory
        overwrite_output_dir=config['training']['overwrite_output_dir'],
        num_train_epochs=config['training']['num_train_epochs'],  # total number of training epochs
        learning_rate=config['training']['learning_rate'], # learning_rate
        per_device_train_batch_size=config['training']['per_device_train_batch_size'], # batch size per device during training
        per_device_eval_batch_size=config['training']['per_device_eval_batch_size'],# batch size for evaluation
        warmup_ratio=config['training']['warmup_ratio'],  # number of warmup steps for learning rate scheduler
        weight_decay=config['training']['weight_decay'],  # strength of weight decay
        lr_scheduler_type=config['training']['lr_scheduler_type'],
        optim =config['training']['optim'],
        gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
        eval_strategy=config['training']['evaluation_strategy'], # evaluation strategy to adopt during training
        save_strategy =config['training']['save_strategy'],
        save_total_limit=config['training']['save_total_limit'], # number of total save model.
        fp16=config['training']['fp16'],
        load_best_model_at_end=config['training']['load_best_model_at_end'], # 최종적으로 가장 높은 점수 저장
        seed=config['training']['seed'],
        logging_dir=config['training']['logging_dir'], # directory for storing logs
        logging_strategy=config['training']['logging_strategy'],
        # predict_with_generate=config['training']['predict_with_generate'], #To use BLEU or ROUGE score
        # generation_max_length=config['training']['generation_max_length'],
        do_train=config['training']['do_train'],
        do_eval=config['training']['do_eval'],
        report_to=config['training']['report_to'], # (선택) wandb를 사용할 때 설정합니다.
        # 필요 시 추가 하이퍼파라미터 설정
        padding_value=tokenizer.pad_token_id,  # explicit padding_value 지정 가능
    )


    # (선택) 모델의 학습 과정을 추적하는 wandb를 사용하기 위해 초기화 해줍니다.
    wandb.init(
        project=config['wandb']['project'],
        name=config['wandb']['name'],
    )

    # (선택) 모델 checkpoint를 wandb에 저장하도록 환경 변수를 설정합니다.
    os.environ["WANDB_LOG_MODEL"]="false"
    os.environ["WANDB_WATCH"]="false"

    # Validation loss가 더 이상 개선되지 않을 때 학습을 중단시키는 EarlyStopping 기능을 사용합니다.
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('-'*10, 'Make training arguments complete', '-'*10,)
    print('-'*10, 'Make trainer', '-'*10,)

    # Trainer 클래스를 정의합니다.
    # trainer = Seq2SeqTrainer(
    #     model=generate_model, # 사용자가 사전 학습하기 위해 사용할 모델을 입력합니다.
    #     args=training_args,
    #     train_dataset=train_inputs_dataset,
    #     eval_dataset=val_inputs_dataset,
    #     compute_metrics = lambda pred: compute_metrics(config,tokenizer, pred),
    #     callbacks = [MyCallback]
    # )
    dpo_trainer = DPOTrainer(
        model=generate_model,
        ref_model=None,  # 비교용 참조모델. 생략 시 자체 복사본
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        compute_metrics = lambda pred: compute_metrics(config,tokenizer, pred),
        callbacks = [MyCallback],
        # 반드시 column mapping 명시 필요
    )
    print('-'*10, 'Make trainer complete', '-'*10,)

    return dpo_trainer

In [12]:
# 학습을 위한 tokenizer와 사전 학습된 모델을 불러옵니다.
def load_tokenizer_and_model_for_train(config,device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)
    print('-'*10, f'Model Name : {config["general"]["model_name"]}', '-'*10,)
    model_name = config['general']['model_name']
    bart_config = BartConfig().from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    generate_model = BartForConditionalGeneration.from_pretrained(config['general']['model_name'],config=bart_config)

    special_tokens_dict={'additional_special_tokens':config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model.resize_token_embeddings(len(tokenizer)) # 사전에 special token을 추가했으므로 재구성 해줍니다.
    generate_model.to(device)
    print(generate_model.config)

    print('-'*10, 'Load tokenizer & model complete', '-'*10,)
    return generate_model , tokenizer

In [13]:

def main(config):
    # 사용할 device를 정의합니다.
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    # 사용할 모델과 tokenizer를 불러옵니다.
    generate_model , tokenizer = load_tokenizer_and_model_for_train(config,device)
    print('-'*10,"tokenizer special tokens : ",tokenizer.special_tokens_map,'-'*10)

    # 학습에 사용할 데이터셋을 불러옵니다.
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token']) # decoder_start_token: str, eos_token: str
    data_path = config['general']['data_path']
    train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config,preprocessor, data_path, tokenizer)
    pprint(train_inputs_dataset)
    pprint(val_inputs_dataset)

    # Trainer 클래스를 불러옵니다.
    trainer = load_trainer_for_train(config, generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset)
    trainer.train()   # 모델 학습을 시작합니다.

    # (선택) 모델 학습이 완료된 후 wandb를 종료합니다.
    wandb.finish()

In [14]:
if __name__ == "__main__":
    main(loaded_config)

---------- device : cuda:0 ----------
2.7.1+cu126
---------- Load tokenizer & model ----------
---------- Model Name : digit82/kobart-summarization ----------


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


BartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 1026,
  "model_type": "bart",
  "normalize_before"

wandb: Currently logged in as: hoppure (hoppure-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


---------- Make training arguments complete ----------
---------- Make trainer ----------


...

Applying chat template to eval dataset:   0%|          | 0/499 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/499 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


---------- Make trainer complete ----------


OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 59.88 MiB is free. Process 924224 has 2.26 GiB memory in use. Process 1049631 has 21.37 GiB memory in use. Of the allocated memory 20.83 GiB is allocated by PyTorch, and 238.31 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [50]:
# 이곳에 내가 사용할 wandb config 설정
loaded_config['inference']['ckt_path'] = "/data/ephemeral/home/dev/code/checkpoint-2000"

- test data를 사용하여 모델의 성능을 확인합니다.

In [51]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_test_dataset(config,preprocessor, tokenizer):

    test_file_path = os.path.join(config['general']['data_path'],'test.csv')

    test_data = preprocessor.make_set_as_df(test_file_path,is_train=False)
    test_id = test_data['fname']

    print('-'*150)
    print(f'test_data:\n{test_data["dialogue"][0]}')
    print('-'*150)

    encoder_input_test , decoder_input_test = preprocessor.make_input(test_data,is_test=True)
    print('-'*10, 'Load data complete', '-'*10,)

    test_tokenized_encoder_inputs = tokenizer(encoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False,)
    test_tokenized_decoder_inputs = tokenizer(decoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False,)

    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-'*10, 'Make dataset complete', '-'*10,)

    return test_data, test_encoder_inputs_dataset

In [52]:
# 추론을 위한 tokenizer와 학습시킨 모델을 불러옵니다.
def load_tokenizer_and_model_for_test(config,device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)

    model_name = config['general']['model_name']
    ckt_path = config['inference']['ckt_path']
    print('-'*10, f'Model Name : {model_name}', '-'*10,)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model = BartForConditionalGeneration.from_pretrained(ckt_path)
    generate_model.resize_token_embeddings(len(tokenizer))
    generate_model.to(device)
    print('-'*10, 'Load tokenizer & model complete', '-'*10,)

    return generate_model , tokenizer

In [53]:
# 학습된 모델이 생성한 요약문의 출력 결과를 보여줍니다.
def inference(config):
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    generate_model , tokenizer = load_tokenizer_and_model_for_test(config,device)

    data_path = config['general']['data_path']
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'])

    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config,preprocessor, tokenizer)
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            generated_ids = generate_model.generate(input_ids=item['input_ids'].to('cuda:0'),
                            no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                            early_stopping=config['inference']['early_stopping'],
                            max_length=config['inference']['generate_max_length'],
                            num_beams=config['inference']['num_beams'],
                        )
            for ids in generated_ids:
                result = tokenizer.decode(ids)
                summary.append(result)

    # 정확한 평가를 위하여 노이즈에 해당되는 스페셜 토큰을 제거합니다.
    remove_tokens = config['inference']['remove_tokens']
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    output = pd.DataFrame(
        {
            "fname": test_data['fname'],
            "summary" : preprocessed_summary,
        }
    )
    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(os.path.join(result_path, "output.csv"), index=False)

    return output

In [54]:
# 학습된 모델의 test를 진행합니다.
if __name__ == "__main__":
    output = inference(loaded_config)

---------- device : cuda:0 ----------
2.7.1+cu126
---------- Load tokenizer & model ----------
---------- Model Name : EbanLee/kobart-summary-v3 ----------


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


---------- Load tokenizer & model complete ----------
------------------------------------------------------------------------------------------------------------------------------------------------------
test_data:
#Person1#: Ms. Dawson, 받아쓰기 좀 부탁드려야겠어요. 
#Person2#: 네, 말씀하세요... 
#Person1#: 이걸 오늘 오후까지 모든 직원들에게 사내 메모로 보내야 해요. 준비됐나요? 
#Person2#: 네, 말씀하세요. 
#Person1#: 모든 직원에게 알립니다... 즉시 발효되어 모든 사내 통신은 이메일과 공식 메모로만 제한됩니다. 근무 시간 동안 즉시 메시지 프로그램 사용은 금지됩니다. 
#Person2#: 이 정책이 사내 통신에만 적용되나요, 아니면 외부 통신에도 해당되나요? 
#Person1#: 이는 모든 통신에 적용됩니다. 사무실 내 직원 간 통신 뿐만 아니라 외부 통신도 해당됩니다. 
#Person2#: 하지만 많은 직원들이 고객과 소통하려고 즉시 메시지를 사용합니다. 
#Person1#: 통신 방법을 바꿔야 할 것입니다. 이 사무실에서는 즉시 메시지를 사용하는 것을 원하지 않습니다. 너무 많은 시간이 낭비됩니다! 이제 계속해서 메모를 작성해 주세요. 어디까지 했죠? 
#Person2#: 내외부 통신에 적용됩니다. 
#Person1#: 네. 즉시 메시지를 계속 사용하면 경고 후 시정 조치가 이루어지며, 두 번째 위반 시 해고될 수 있습니다. 이번 정책에 관한 질문은 부서장에게 문의하세요. 
#Person2#: 그게 다인가요? 
#Person1#: 네. 오늘 오후 4시까지 이 메모를 작성하고 배포해주세요.
----------------------------------------------------------------------------

100%|██████████| 16/16 [00:42<00:00,  2.68s/it]


In [21]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

,fname,summary
0,test_0,Ms. Dawson은 이메일과 공식메시지 사용이 금지되는 것을 알리기 위해 메모...
1,test_1,Carrefour 교차로 근처에서 교통체증이 심해지자 대중교통으로 출퇴근할 것을...
2,test_2,"Masha와 Hero는 두 달 동안 별거 중이기에 이혼을 하기로 결정하였으나, ..."
3,test_3,Brian은 생일 축하 파티를 즐기기 위해 함께 한 잔 할 예정이다. ...
4,test_4,모금 은 올림픽 스타디움에 대해 이야기합니다. ...
...,...,...
494,test_495,Jack은 Charlie의 새로운 게임을 샀다. ...
495,test_496,라디오 방송국에서 일하게 된 그는 CBC에 제안하여 Golden Country ...
496,test_497,Alice는 세탁기를 사용하지만 기계에 비누가 들어 있지 않아 불편함을 호소합니...
497,test_498,Mateve는 Mrs에게 아파트를 보여줄 수 있는지 물어보고 다시 연락할 예정이...


In [29]:
output

,fname,summary
0,test_0,"Ms와 Dawson은 이메일과 공식메시지 사용이 금지되며, 메모를 작성하고 배포..."
1,test_1,Carrefour 교차로 근처에서 교통체증이 심해지자 출퇴근을 위해 대중교통을 ...
2,test_2,Masha와 Hero가 두 달 동안 별거 중이더니 결국 이혼을 결정했다. ...
3,test_3,Brian은 생일 축하 파티를 즐기기 위해 파티에 참석할 예정이다. ...
4,test_4,올림픽 스타디움은 이번 6월에 완공될 예정이다. ...
...,...,...
494,test_495,Charlie는 숙제만 먼저 끝내면 된다. ...
495,test_496,"라디오 방송국에서 일하게 된 프로그램은 CBC에 제안하러 갔고, Golde..."
496,test_497,Alice는 기계가 비누를 따로 넣지 않아 세탁기에 익숙하지 않다. ...
497,test_498,Matthew와 Thou는 Mrs에게 언제 아파트를 보여줄 수 있는지 물어보고 ...
